On this project i will build a CNN, that can make a difference between a Cat and a Dog. This is a project is made to teach me some methods of pooling and flattening that the CNN methods use.

# Libraries

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D # first step of convolution to build the feature mapping
from keras.layers import MaxPooling2D # to maxpool.
from keras.layers import Flatten # step3 to flatten our matrix.
from keras.layers import Dense # to add fully connected layers.

Using TensorFlow backend.


# CNN BUILD

## Initalisation

It's just like a simple ANN to initalize, we build de classifier

In [2]:
classifier=Sequential()

## Convolution

after that we build the convolution layer. This is the layer where we build our feature maps, by translating our input img , into multiple matrixs.
By using Convolution2D rather than Dense, to build our feature maps.
- filter: the number of features we want. (32 is a standard on the first layer) if we make another convolutional layer we would double the filter value.
- kernel_size: the matrix dimension of the feature detector, in our case we choose a 3x3 matrix. so a simple 3 value.
- stride: the amount of pixel we move the feature detector.

- input_shape: to specify the shape of our imgs, (width,height,3) if RGB and (width,height,1) if B&W. We will adapt our imgs to the specified width and height. In our case (64,64,3).
- activation: the activation function we use "relu" because handling imgs is non linear, which makes relu a better option


In [3]:
classifier.add(Convolution2D(filters=32,kernel_size=3,strides=1,input_shape=(64,64,3),activation="relu"))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Pooling

In this step we reduce every feature map, into a smaller matrix, to make it easier to process. Creating the so called Pooling Layer. if we don't do it we will have WAY TO MUCH features.
- pool_size: the dimension of the 'feature detector' in our case 2x2 . the strides will have the default value of 2 in this case.

In [4]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

## (added a second convolutional layer,to make better prediction)

in this case we don't need input_shape, because we already adapted our imgs the first time.

In [5]:
classifier.add(Convolution2D(filters=32,kernel_size=3,strides=1,activation="relu"))

we add a MAXPOOL for this case too.

In [6]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

## Flattening

We take all our pooled feature maps and align them to make the Entry layer for our ANN. Since now the structures of the img are represented as a high number, we can identify them as features for our entry layer.

In [7]:
classifier.add(Flatten())

Now we just need to build a simple ANN.

## ANN build.

We create a hidden layer. I choose the units=128 because we need high enough value for the amount of features.

In [8]:
classifier.add(Dense(units=128,activation="relu"))

We build now the exit layer. Since this project is a classification of 2 possible outcomes, it would be wise to use the sigmoid activation function. (if we had multiple outcomes we could use the softmax function.)

In [9]:
classifier.add(Dense(units=1,activation="sigmoid"))

now we compile the classifier.

In [10]:
classifier.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# FIT the CNN to our imgs.

we need to fit the cnn to our imgs. We will use a technique that is called img augmentation, it prepares our imgs to reduce the risk of OVERFITTING to nothingness. Because to train a model, wee need a lot of imgs, this technique creates new imgs based on our current imgs.(for example just turn a img upside down.) The function is called : Flow_from_directory() we copy the functions from the keras library.

In [11]:
from keras.preprocessing.image import ImageDataGenerator

the steps per epoch is the value of the imgs divided by the bach size : wich makes in our case 8000 / 32 = 250, so we give it 250. for validation step we have 2000 imgs divided by 32 == 63.

In [12]:
train_datagen = ImageDataGenerator(
        rescale=1./255, # change the scale value of every  (just like the standardization)
        shear_range=0.2,# to make transvection = change the angle of which you see the img
        zoom_range=0.2,
        horizontal_flip=True) # to turn the img horizontaly.

test_datagen = ImageDataGenerator(rescale=1./255) #we change everything on the same scale.

# creates the new imgs.

training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 8000 images belonging to 2 classes.


for unknown reasons in google colab we find only 3270 images

In [13]:
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 2000 images belonging to 2 classes.


for unknown reasons in google colab we find 0 images

In [14]:
# trains the model and measures his performance.
classifier.fit(
        training_set,
        steps_per_epoch=250, #everytime we reajust the epochs
        epochs=25,
        validation_data=test_set,
        validation_steps=63)


Epoch 1/25
250/250 [==============================] - 25s 101ms/step - loss: 0.6690 - accuracy: 0.5971 - val_loss: 0.6366 - val_accuracy: 0.6125
Epoch 2/25
250/250 [==============================] - 24s 95ms/step - loss: 0.6209 - accuracy: 0.6624 - val_loss: 0.4129 - val_accuracy: 0.6915
Epoch 3/25
250/250 [==============================] - 24s 98ms/step - loss: 0.5774 - accuracy: 0.7009 - val_loss: 0.3902 - val_accuracy: 0.6980
Epoch 4/25
250/250 [==============================] - 25s 100ms/step - loss: 0.5446 - accuracy: 0.7226 - val_loss: 0.6473 - val_accuracy: 0.7235
Epoch 5/25
250/250 [==============================] - 31s 124ms/step - loss: 0.5085 - accuracy: 0.7487 - val_loss: 0.5678 - val_accuracy: 0.7325
Epoch 6/25
250/250 [==============================] - 32s 127ms/step - loss: 0.4869 - accuracy: 0.7635 - val_loss: 0.9314 - val_accuracy: 0.7675
Epoch 7/25
250/250 [==============================] - 32s 129ms/step - loss: 0.4778 - accuracy: 0.7717 - val_loss: 0.3704 - val_acc

this takes a long time....(first run without the second convolutional layer) 0.76 is not very high in my point of view, so i will add some convolutional layers... (ps at first i used only one convolutional layer, now i use 2.)

# Test on a new image

In [15]:
classifier.predict("dataset/single_prediction")

AttributeError: 'str' object has no attribute 'ndim'